In [ ]:
!pip install mediapipe

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt

def detect_and_save_hand(image_path, output_path, img_size=400, padding=50):
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    with mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5) as hands:
        results = hands.process(image_rgb)
        
        if not results.multi_hand_landmarks:
            print("No hands detected in", image_path)
            return
        
        landmarks = results.multi_hand_landmarks[0]
        h, w, _ = image.shape
        
        # Extract landmark coordinates
        points = [(int(l.x * w), int(l.y * h)) for l in landmarks.landmark]
        
        # Get bounding box of hand
        x_min, y_min = np.min(points, axis=0)
        x_max, y_max = np.max(points, axis=0)
        
        # Expand bounding box
        x_min, y_min = max(0, x_min - padding), max(0, y_min - padding)
        x_max, y_max = min(w, x_max + padding), min(h, y_max + padding)
        
        # Create a blank white image
        hand_img = np.ones((img_size, img_size, 3), dtype=np.uint8) * 255
        
        # Scale and center hand skeleton
        hand_width, hand_height = x_max - x_min, y_max - y_min
        scale = (img_size - 2 * padding) / max(hand_width, hand_height)
        offset_x = (img_size - hand_width * scale) // 2
        offset_y = (img_size - hand_height * scale) // 2
        
        scaled_points = [
            (int((x - x_min) * scale + offset_x), int((y - y_min) * scale + offset_y))
            for x, y in points
        ]
        
        # Draw hand skeleton
        for connection in mp_hands.HAND_CONNECTIONS:
            x1, y1 = scaled_points[connection[0]]
            x2, y2 = scaled_points[connection[1]]
            cv2.line(hand_img, (x1, y1), (x2, y2), (0, 0, 0), 2)
        
        # Draw hand landmarks
        for x, y in scaled_points:
            cv2.circle(hand_img, (x, y), 2, (0, 0, 255), -1)
        
        cv2.imwrite(output_path, hand_img)

# Example usage
detect_and_save_hand("/kaggle/input/rgb-arabic-alphabets-sign-language-dataset/RGB ArSL dataset/Ghain/Ghain_0.jpg", "output.png")


# Preprocessing

In [ ]:
import pandas as pd
import os
import cv2
import mediapipe as mp
from tqdm import tqdm
import sys

In [ ]:
folders = [
    "Alef", "Beh", "Teh", "Theh", "Jeem", "Hah", "Khah", "Dal", 
    "Thal", "Reh", "Zain", "Seen", "Sheen", "Sad", "Dad", "Tah", 
    "Zah", "Ain", "Ghain", "Feh", "Qaf", "Kaf", "Lam", "Meem", 
    "Noon", "Heh", "Waw", "Yeh", "Laa", "Teh_Marbuta", "Al"
]

EnAr = {
    "Alef": "أ", "Beh": "ب", "Teh": "ت", "Theh": "ث",
    "Jeem": "ج", "Hah": "ح", "Khah": "خ", "Dal": "د",
    "Thal": "ذ", "Reh": "ر", "Zain": "ز", "Seen": "س",
    "Sheen": "ش", "Sad": "ص", "Dad": "ض", "Tah": "ط",
    "Zah": "ظ", "Ain": "ع", "Ghain": "غ", "Feh": "ف",
    "Qaf": "ق", "Kaf": "ك", "Lam": "ل", "Meem": "م",
    "Noon": "ن", "Heh": "ه", "Waw": "و", "Yeh": "ي",
    "Laa": "لا", "Teh_Marbuta": "ة", "Al": "ال"
}


In [ ]:
def detect_hand(image_path):
    mp_hands = mp.solutions.hands
    
    # Read image
    image = cv2.imread(image_path)
    if image is None:
        return 0  

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Suppress MediaPipe logs by redirecting stderr
    with open(os.devnull, "w") as f, mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5) as hands:
        stderr_original = sys.stderr  # Save original stderr
        sys.stderr = f  # Redirect stderr
        results = hands.process(image_rgb)
        sys.stderr = stderr_original  # Restore stderr
    
    return 1 if results.multi_hand_landmarks else 0

In [ ]:
dataset = pd.DataFrame(columns=['Folder', 'Path', 'Hands'])

In [ ]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

dataset_path = "/kaggle/input/rgb-arabic-alphabets-sign-language-dataset/RGB ArSL dataset"

for folder in folders:
    path = f"{dataset_path}/{folder}"  

    # Get list of image files
    image_files = !ls "{path}" | grep -E '\.jpg$|\.jpeg$|\.png$|\.JPG$|\.JPEG$'
    
    if not image_files:
        continue  
    
    total_files = len(image_files)

    # Initialize progress bar with clean format
    pbar = tqdm(total=total_files, desc=f"{folder}: ", unit="", bar_format="{desc}{percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt}", ncols=50)
    
    for image_file in image_files:
        image_path = f"{path}/{image_file}"
        HandChecked = detect_hand(image_path)
        new_row = pd.DataFrame([{'Folder': folder, 'Path': image_path, 'Hands': HandChecked}])
        dataset = pd.concat([dataset, new_row], ignore_index=True)

        pbar.update(1)  # Update progress bar
    
    pbar.close()



In [ ]:
dataset.describe()

In [ ]:
dataset.to_csv("/kaggle/working/hand_detection_results.csv", index=False)
print("✅ CSV file saved: /kaggle/working/hand_detection_results.csv")

In [ ]:
summary_text = dataset.describe().to_string()  # Convert DataFrame summary to text

with open("/kaggle/working/dataset_summary.txt", "w") as f:
    f.write(summary_text)

print("✅ Dataset summary saved: /kaggle/working/dataset_summary.txt")